In [1]:
# Author: Nicholas Benavides
# Code liberally inspired by and lifted from:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/bidirectional_recurrent_neural_network/main.py

In [326]:
import torch 
import torch.nn as nn
from torch.utils import data
import numpy as np
from sklearn.metrics import classification_report

In [11]:
# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
elmo_X = np.load('pol-balanced-elmo-X.npy')
elmo_y = np.load('pol-balanced-elmo-y.npy')

In [38]:
train_data = []
for i in range(len(elmo_X)):
   train_data.append([elmo_X[i], elmo_y[i]])
print(len(train_data))
print(len(train_data[0]))
print(len(train_data[0][0]))

6834
2
2048


In [265]:
# Hyper-parameters
sequence_length = 1
input_size = 2048
hidden_size = 128
num_layers = 2
num_classes = 2
batch_size = 100
num_epochs = 2
learning_rate = 0.003

In [266]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [309]:
# Bidirectional recurrent neural network (many-to-one)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)#.to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size)#.to(device)
        # Forward propagate LSTM
        #print(x.shape)
        x = x.unsqueeze(0)
        #print(x.shape)
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        #print(out.shape)
        # Decode the hidden state of the last time step
        out = self.fc(out[-1, :, :])
        #print(out.shape)
        return out

In [310]:
model = BiRNN(input_size, hidden_size, num_layers, num_classes)#.to(device)

In [311]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [312]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (sequences, labels) in enumerate(train_loader):
        
        #print(sequences.shape)
        sequences = sequences.view(sequences.shape[0], input_size)#.to(device)
        labels = labels#.to(device)
        
        # Forward pass
        outputs = model(sequences)
        #print(outputs.shape)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


In [331]:
with torch.no_grad():
    correct = 0
    total = 0
    preds = []
    actuals = []
    for sequences, labels in test_loader:
        sequences = sequences.view(sequences.shape[0], input_size)#.to(device)
        labels = labels#.to(device)
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, 1)
        preds.append(predicted.tolist())
        actuals.append(labels.tolist())

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

In [332]:
preds = [item for sublist in preds for item in sublist]
actuals = [item for sublist in actuals for item in sublist]
print(classification_report(actuals, preds, digits=3))


              precision    recall  f1-score   support

           0      0.815     0.768     0.791      3438
           1      0.778     0.823     0.800      3396

   micro avg      0.796     0.796     0.796      6834
   macro avg      0.797     0.796     0.796      6834
weighted avg      0.797     0.796     0.796      6834

